# [Research Request - 5311 Apportionment Formula #1710](https://github.com/cal-itp/data-analyses/issues/1710)

- Do the past 3 years of 5311 apportionments to regional bodies passing through Caltrans align with the stated formula? If not, how would apportionments change? How would apportionments change if allocations went directly to each 5311-recipient agency?


- Double check work done by Sara.

- Cross reference all available types of census data.

- can we use the census API to read in data to a notebook? AND apple the apportionment formula

Notes from Grant team 5311 apportionment excel sheet
1. Metropolitan Transportation Commission (MTC): 
  - Sonoma, 
  - Napa, 
  - Solano, 
  - Contra Costa, 
  - Alameda, 
  - Marin, 
  - San Francisco, 
  - San Mateo and 
  - Santa Clara
2. Sacramento Area Council of Governments (SACOG): 
  - Yuba, 
  - Sutter, 
  - Yolo and 
  - Sacramento
3. Alpine and Sierra Counties (collectively, 'Minimum Counties') receive a minimum funding amount of $48,000 due to low population (under 7,590)
4. Orange County does not receive Rural funds


In [26]:
import pandas as pd
import altair as alt

## Read in Census Datasets

### API data

In [2]:
# API_Data = "https://api.census.gov/data/2020/dec/cd118?get=group(P2)&ucgid=pseudo(0100000US$0400000)"

In [3]:
# df = pd.read_json(API_Data)

In [4]:
# df.head()

### State-level Urban and Rural Information for the 2020 Census and 2010 Census
https://www.census.gov/programs-surveys/geography/guidance/geo-areas/urban-rural.html

In [5]:
# state_rural_url = "https://www2.census.gov/geo/docs/reference/ua/State_Urban_Rural_Pop_2020_2010.xlsx"

# r = requests.get(state_rural_url)



# with open("state_rural.xlsx", "wb") as f:
#     f.write(r.content)

# state_rural_data = pd.read_excel(state_rural_url) # getting error

# display(
#     state_rural_data.info(),
#     state_rural_data.head()
# )

### County-level Urban and Rural information for the 2010 & 2020 Census

In [6]:
county_rural_2020_url = "https://www2.census.gov/geo/docs/reference/ua/2020_UA_COUNTY.xlsx"

county_rural_2010_url = "https://www2.census.gov/geo/docs/reference/ua/PctUrbanRural_County.xls"

# read in data
county_rural_data_2020 = pd.read_excel(county_rural_2020_url)
county_rural_data_2010 = pd.read_excel(county_rural_2010_url)

# lower case the columns
county_rural_data_2020.columns = county_rural_data_2020.columns.str.lower()
county_rural_data_2010.columns = county_rural_data_2010.columns.str.lower()

# add suffix to distinuish years
county_rural_data_2020 = county_rural_data_2020.add_suffix("_2020")
county_rural_data_2010 = county_rural_data_2010.add_suffix("_2010")

In [7]:
display(
    county_rural_data_2020.head(),
    county_rural_data_2010.head(),
)

,state_2020,county_2020,state_name_2020,county_name_2020,pop_cou_2020,hou_cou_2020,aland_cou_2020,aland_mi²_cou_2020,popden_cou_2020,houden_cou_2020,...,pop_rur_2020,poppct_rur_2020,hou_rur_2020,houpct_rur_2020,aland_rur_2020,aland_mi²_rur_2020,aland_pct_rur_2020,popden_rur_2020,houden_rur_2020,ruralblocks_2020
0,1,1,Alabama,Autauga,58805,24350,1539634184,594.452758,98.922916,40.962044,...,23920,0.406768,9991,0.410308,1483727020,572.867002,0.963688,41.754892,17.440348,991
1,1,3,Alabama,Baldwin,231767,124148,4117656199,1589.827058,145.781265,78.088997,...,87113,0.375865,40740,0.328157,3762600021,1452.739868,0.913772,59.964624,28.043562,3181
2,1,5,Alabama,Barbour,25223,11618,2292160149,885.003034,28.500467,13.127639,...,16627,0.659200,7538,0.648821,2276027730,878.774307,0.992962,18.920672,8.577857,1011
3,1,7,Alabama,Bibb,22293,9002,1612188717,622.466064,35.814001,14.461833,...,22293,1.000000,9002,1.000000,1612188717,622.466064,1.000000,35.814001,14.461833,1090
4,1,9,Alabama,Blount,59134,24622,1670259090,644.887035,91.696680,38.180330,...,53510,0.904894,22337,0.907197,1658933117,640.514076,0.993219,83.542270,34.873551,2207


,state_2010,county_2010,statename_2010,countyname_2010,pop_cou_2010,area_cou_2010,pop_urban_2010,poppct_urban_2010,area_urban_2010,areapct_urban_2010,...,pop_uc_2010,poppct_uc_2010,area_uc_2010,areapct_uc_2010,popden_uc_2010,pop_rural_2010,poppct_rural_2010,area_rural_2010,areapct_rural_2010,popden_rural_2010
0,1,1,Alabama,Autauga,54571,1539582278,31650,58.00,50882539,3.30,...,0,0.00,0,0.00,NaN,22921,42.00,1488699739,96.70,39.9
1,1,3,Alabama,Baldwin,182265,4117521611,105205,57.72,275001368,6.68,...,41556,22.80,111381848,2.71,966.3,77060,42.28,3842520243,93.32,51.9
2,1,5,Alabama,Barbour,27457,2291818968,8844,32.21,17447656,0.76,...,8844,32.21,17447656,0.76,1312.8,18613,67.79,2274371312,99.24,21.2
3,1,7,Alabama,Bibb,22915,1612480789,7252,31.65,18903236,1.17,...,7252,31.65,18903236,1.17,993.6,15663,68.35,1593577553,98.83,25.5
4,1,9,Alabama,Blount,57322,1669961855,5760,10.05,10874700,0.65,...,5374,9.38,9851721,0.59,1412.8,51562,89.95,1659087155,99.35,80.5


### filter for just California

In [8]:
ca_county_2010 = county_rural_data_2010[county_rural_data_2010["statename_2010"]=="California"]
ca_county_2020 = county_rural_data_2020[county_rural_data_2020["state_name_2020"]=="California"]

# ca_county_2010["countyname_2010"].nunique() == ca_county_2020["county_name_2020"].nunique() #True

True

### aggregate by county name, sum population columns

In [11]:
county_2010 = ca_county_2010.groupby("countyname_2010").agg(
    {"pop_rural_2010":"sum", 
     "pop_cou_2010":"sum", 
     "poppct_rural_2010":"sum"}).reset_index()

county_2020 = ca_county_2020.groupby("county_name_2020").agg(
    {"pop_rur_2020":"sum", 
     "pop_cou_2020":"sum", 
     "poppct_rur_2020":"sum"}).reset_index()

# round population %
county_2020["poppct_rur_2020"] = (county_2020["poppct_rur_2020"]*100).round(2)

In [12]:
display(
    county_2010.head(),
    county_2020.head()
)

,countyname_2010,pop_rural_2010,pop_cou_2010,poppct_rural_2010
0,Alameda,5869,1510271,0.39
1,Alpine,1175,1175,100.00
2,Amador,23016,38091,60.42
3,Butte,41584,220000,18.90
4,Calaveras,34370,45578,75.41


,county_name_2020,pop_rur_2020,pop_cou_2020,poppct_rur_2020
0,Alameda,8253,1682353,0.49
1,Alpine,1204,1204,100.00
2,Amador,28020,40474,69.23
3,Butte,44478,211632,21.02
4,Calaveras,37128,45292,81.97


### merge 2010 and 2020 county census data

In [65]:
merge = county_2010.merge(
    county_2020,
    left_on = "countyname_2010",
    right_on = "county_name_2020",
    how= "inner",
    indicator= True
).drop(columns= ["county_name_2020","pop_cou_2010","pop_cou_2020","poppct_rural_2010", "poppct_rur_2020","_merge"])

### consolidate specific counties to MTC and SACOG

In [68]:
mtc = [
    "Sonoma", 
    "Napa", 
    "Solano", 
    "Contra Costa", 
    "Alameda", 
    "Marin", 
    "San Francisco", 
    "San Mateo", 
    "Santa Clara"
]

sacog = [
    "Yuba", 
    "Sutter", 
    "Yolo", 
    "Sacramento"
]

# add MTC row
mtc_row = merge[merge["countyname_2010"].isin(mtc)][["pop_rural_2010","pop_rur_2020"]].sum()

mtc_row["countyname_2010"] = "MTC"

merge = pd.concat([
    merge,
    pd.DataFrame([mtc_row])],
    ignore_index=True
)

# add SACOG row
sacog_row = merge[merge["countyname_2010"].isin(sacog)][["pop_rural_2010","pop_rur_2020"]].sum()

sacog_row["countyname_2010"] = "SACOG"

merge = pd.concat([
    merge,
    pd.DataFrame([sacog_row])],
    ignore_index=True
)

# remove individual mtc and sacog counties
merge = merge[(~merge["countyname_2010"].isin(mtc))
     &(~merge["countyname_2010"].isin(sacog))
]

## Applying OTGC formula
- Sara's initial formula 

apportionment = (county rural population/ total net population) * total apportionment available to counties - 197.27


In [70]:
# https://www.transit.dot.gov/funding/apportionments/table-9-fy-2025-section-5311-and-section-5340-rural-area-formula-0
ca_total_5311_apportionment = 43540762 

# total after removing (admin, min county, 5311f stuff)
total_available_to_counties = 32559572 

# list of min counties 
excluded_county_pop = ["Alpine", "Orange", "Sierra"]

# calculate net population
net_pop_2010 = merge["pop_rural_2010"].sum()-(merge[merge["countyname_2010"].isin(excluded_county_pop)]["pop_rural_2010"].sum())
net_pop_2020 = merge["pop_rur_2020"].sum()-(merge[merge["countyname_2010"].isin(excluded_county_pop)]["pop_rur_2020"].sum())

# function to replicate formula
def otgc_formula(
    rural_pop: int,
    total_net_pop: int,
    total_county_funds:int
):
    return ((rural_pop/total_net_pop)*total_county_funds)-197.27

# calculate 2010 apportionment
merge["apportionment_calc_2010"] = merge.apply(
    lambda row: otgc_formula(
        rural_pop = row["pop_rural_2010"],
        total_net_pop = net_pop_2010,
        total_county_funds = total_available_to_counties
    ), axis=1
).astype("int64")

# calculate 2020 apportionment
merge["apportionment_calc_2020"] = merge.apply(
    lambda row: otgc_formula(
        rural_pop = row["pop_rur_2020"],
        total_net_pop = net_pop_2020,
        total_county_funds = total_available_to_counties
    ), axis=1
).astype("int64")

## manual overwrite for orange, sierra, alpine

In [77]:
min_county_edit = {
    "Alpine":48000,
    "Orange":0,
    "Sierra":48000
}

for county, amount in min_county_edit.items():
    merge.loc[merge["countyname_2010"] == county, "apportionment_calc_2020"] = amount
    merge.loc[merge["countyname_2010"] == county, "apportionment_calc_2010"] = amount

## re-order columns

In [81]:
merge = merge[[
    "countyname_2010",
    "pop_rural_2010",
    "apportionment_calc_2010",
    "pop_rur_2020",
    "apportionment_calc_2020"
]]

In [80]:
# final cleaned dataframe
merge

,countyname_2010,pop_rural_2010,apportionment_calc_2010,pop_rur_2020,apportionment_calc_2020
1,Alpine,1175,48000,1204,48000
2,Amador,23016,400197,28020,402349
3,Butte,41584,723213,44478,638791
4,Calaveras,34370,597716,37128,533198
5,Colusa,6795,118011,9326,133783
7,Del Norte,9634,167399,12123,173966
8,El Dorado,62827,1092765,70112,1007059
9,Fresno,100537,1748782,113625,1632183
10,Glenn,11494,199756,11917,171007
11,Humboldt,40062,696736,43033,618031


## Visuals

### melt dataframe

In [82]:
melt = merge.melt(
    id_vars = ["countyname_2010"],
    value_vars = [
        "pop_rural_2010",
        "apportionment_calc_2010",
        "pop_rur_2020",
        "apportionment_calc_2020"
    ],
    value_name = "metric"
)

melt.sort_values(by="countyname_2010").head()

,countyname_2010,variable,metric
0,Alpine,pop_rural_2010,1175
47,Alpine,apportionment_calc_2010,48000
141,Alpine,apportionment_calc_2020,48000
94,Alpine,pop_rur_2020,1204
142,Amador,apportionment_calc_2020,402349


### 5311 Apportionments per County ($)

In [111]:
alt.Chart(melt[melt["variable"].isin(["apportionment_calc_2010","apportionment_calc_2020"])]).mark_bar().encode(
    x = "countyname_2010:N",
    y = "metric:Q",
    xOffset="variable:N",
    color = "variable:N",
    tooltip = ["countyname_2010","variable","metric"]
).properties(
    width="container",
    title= "5311 Apportionments per County ($)"
)


alt.Chart(...)

### Rural Populations per County

In [109]:
alt.Chart(melt[~melt["variable"].isin(["apportionment_calc_2010","apportionment_calc_2020"])]).mark_bar().encode(
    x = "countyname_2010:N",
    y = "metric:Q",
    xOffset="variable:N",
    color = "variable:N",
    tooltip = ["countyname_2010","variable","metric"]
).properties(
    width= "container",
    title= "Rural Populations per County"
)


alt.Chart(...)

### 5311 Apportionments ($) and Rural Populations per County

In [112]:
alt.Chart(melt).mark_bar().encode(
    x = "variable",
    y = alt.Y("metric:Q").stack(None),
    row="countyname_2010:N",
    color = "variable:N",
    tooltip = ["countyname_2010","variable","metric"]
).properties(
    width=600,
    title= "5311 Apportionments ($) and Rural Populations per County"
).resolve_scale(y="independent")



alt.Chart(...)